In [1]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow --user

  Using cached tensorflow-2.5.0-cp37-cp37m-win_amd64.whl (422.6 MB)
Processing c:\users\dell\appdata\local\pip\cache\wheels\3f\e3\ec\8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2\termcolor-1.1.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached numpy-1.19.5-cp37-cp37m-win_amd64.whl (13.2 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
Processing c:\users\dell\appdata\local\pip\cache\wheels\62\76\4c\aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6\wrapt-1.12.1-py3-none-any.whl
  Using cached keras_nightly-2.5.0.dev2021032900-py2.py3-none-any.whl (1.2 MB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached h5py-3.1.0-cp37-cp37m-win_amd64.whl (2.7 MB)
  Using cached grpcio-1.34.1-cp37-cp37m-win_amd64.whl (2.9 MB)
  Using cached tensorflow_estimator-2.5.0-py2.py3-none-any.whl (462 kB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached tensorboard-2.5.0-py3-none-any.whl (6.0 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorboard_plugi

In [4]:
pip install pydot_ng

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install pydot

In [2]:
"""
    Import all the important required Deep Learning Libraries to train the emotions model.
    
    Keras is an Application Programming Interface (API) which can run on top of tensorflow.
    
    tensorflow will be the main deep learning module we will use to build our deep learning model.
    
    The ImageDataGenerator is used for Data augmentation where the model can see more copies of 
        the model. Data Augmentation is used for creating replications of the original images 
        and using those transformations in each epoch.
        
    The layers for training which will be used are as follows:
    1. Input = The input layer which we pass the input shape.
    2. Conv2D = The Convoluional layer combined with Input to provide a output of tensors
    3. Maxpool2D = Downsampling the Data from the convolutional layer.
    4. Batch normalization = It is a technique for training very deep neural networks that standardizes 
       the inputs to a layer for each mini-batch. This has the effect of stabilizing the learning 
       process and dramatically reducing the number of training epochs required to train deep networks.
    5. Dropout = Dropout is a technique where randomly selected neurons are ignored during training. 
                 They are “dropped-out” randomly and this prevents over-fitting.
    6. Dense = Fully Connected layers.
    7. Flatten = Flatten the entire structure to a 1-D array.
    
    The Models can be built in a model like structure as shown in this particular model or can be built 
        in a sequential manner.
        
    Use of l2 regularization for fine tuning.
    
    The optimizer used will be Adam as it is performs better than the other optimizers on this model.
    
    Numpy for numerical array like operations.
    
    pydot_ng and graphviz are used for making plots.
    
    We are also importing the os module to make it compatible with the windows environment. 
    
"""

import keras
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense, Flatten, Activation
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import pydot_ng
import graphviz
import os

In [3]:
"""
    num_classes = Defines the number of classes we have to predict which are namely 
    Angry, Fear, Happy, Neutral, Surprise, Neutral and Disgust.
    
    From the exploratory Data Analysis we know that The Dimensions of the image are:
    Image Height = 48 pixels
    Image Width = 48 pixels
    Number of channels = 1 because it is a grayscale image.
    
    We will consider a batch size for the training of the image augmentation.

"""

num_classes = 7
batch_size = 64
epochs = 100
Img_height = 48
Img_width = 48

In [4]:
"""
    Store the csv file in data and then print the first 5 rows.

"""

data = pd.read_csv('C:/Users/dell/Desktop/PROJECT DATA/fer2013.csv')
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [5]:
"""
    Reference: https://github.com/oarriaga/face_classification
    We will convert the pixels to list in this method. 
    We split the data by spaces and then take them as arrays and 
    reshape into 48, 48 shape. Then we expand the dimensions 
    and then convert the labels to categorical matrix.

"""

pixels = data['pixels'].tolist() 
faces = []

for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(Img_height, Img_width) 
    faces.append(face.astype('float32'))

faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)

emotions = pd.get_dummies(data['emotion']).values

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

In [7]:
"""
    We will be using a sequential type architecture for our model.
    Our Sequential model will have a total of 5 blocks i.e. three
    convolutional blocks, one fully connected layer and one output
    layer.
    
    We will have 3 Convolutional Blocks with filters of increasing size
    as 32, 64 and 128 respectively. The kernel_size will be (3,3) and
    the kernel_initializer will be he_normal. We can also use a 
    kernel_regularizer with l2 normalization. Our Preferred choice of 
    activation is relu because it usually performs better on most datasets.
    The Input shape will be the same as the size of each of our train and
    validation images.
    The BatchNormalization layer Batch normalization is a technique for improving 
    the speed, performance, and stability of artificial neural networks.
    and Maxpooling is used to Downsample the data. 
    The Dropout layer is used for prevention of overfitting.
    
    The fully connected block consists of a Dense layer of 64 filters
    and a batch normalization followed by a dropout layer. Before passing
    through the Dense layer the data is flattened to match the dimensions.
    
    Finally the output layer consists of a Dense layer with a softmax
    activation to give probabilites according to the num_classes which
    represents the number of predictions to be made. 

"""


model = Sequential()

# Block-1: The First Convolutional Block

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal',
                 activation="relu", 
                 input_shape=(Img_height, Img_width, 1), 
                 name="Conv1"))

model.add(BatchNormalization(name="Batch_Norm1"))

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal', 
                 activation="relu", name="Conv2"))

model.add(BatchNormalization(name="Batch_Norm2"))
model.add(MaxPooling2D(pool_size=(2,2), name="Maxpool1"))
model.add(Dropout(0.5, name="Dropout1"))

# Block-2: The Second Convolutional Block

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal',
                 activation="relu", name="Conv3"))

model.add(BatchNormalization(name="Batch_Norm3"))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',
                 kernel_initializer='he_normal', 
                 activation="relu", name="Conv4"))

model.add(BatchNormalization(name="Batch_Norm4"))
model.add(MaxPooling2D(pool_size=(2,2), name="Maxpool2"))
model.add(Dropout(0.5, name="Dropout2"))

# Block-3: The Third Convolutional Block

model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal', 
                 activation="relu", name="Conv5"))

model.add(BatchNormalization(name="Batch_Norm5"))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', 
                 kernel_initializer='he_normal',
                 activation="relu", name="Conv6"))

model.add(BatchNormalization(name="Batch_Norm6"))
model.add(MaxPooling2D(pool_size=(2,2), name="Maxpool3"))
model.add(Dropout(0.5, name="Dropout3"))

# Block-4: The Fully Connected Block

model.add(Flatten(name="Flatten"))
model.add(Dense(64, activation="relu", kernel_initializer='he_normal', name="Dense"))
model.add(BatchNormalization(name="Batch_Norm7"))
model.add(Dropout(0.5, name="Dropout4"))

# Block-5: The Output Block

model.add(Dense(num_classes, activation="softmax", kernel_initializer='he_normal', name = "Output"))

In [8]:


model.summary()



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 48, 48, 32)        320       
_________________________________________________________________
Batch_Norm1 (BatchNormalizat (None, 48, 48, 32)        128       
_________________________________________________________________
Conv2 (Conv2D)               (None, 48, 48, 32)        9248      
_________________________________________________________________
Batch_Norm2 (BatchNormalizat (None, 48, 48, 32)        128       
_________________________________________________________________
Maxpool1 (MaxPooling2D)      (None, 24, 24, 32)        0         
_________________________________________________________________
Dropout1 (Dropout)           (None, 24, 24, 32)        0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 24, 24, 64)        1

In [10]:
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from keras.utils import np_utils

keras.utils.plot_model(model, to_file='model3.png', show_layer_names=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [11]:


model.compile(loss="categorical_crossentropy",
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])



C:\Users\dell\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

tensorboard = TensorBoard(log_dir='logs3')
lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.9, patience=3, verbose=1)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=8, verbose=1, mode='auto')
checkpointer = ModelCheckpoint("emotions3.h5", monitor='val_accuracy', verbose=1, save_best_only=True)

In [13]:
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test)),
          shuffle=True,
          callbacks=[lr_reducer, tensorboard, early_stopper, checkpointer])

Epoch 1/100
455/455 [==============================] - 260s 563ms/step - loss: 2.0942 - accuracy: 0.2444 - val_loss: 1.7046 - val_accuracy: 0.3123

Epoch 00001: val_accuracy improved from -inf to 0.31234, saving model to emotions3.h5
Epoch 2/100
455/455 [==============================] - 278s 610ms/step - loss: 1.6894 - accuracy: 0.3624 - val_loss: 1.4580 - val_accuracy: 0.4355

Epoch 00002: val_accuracy improved from 0.31234 to 0.43550, saving model to emotions3.h5
Epoch 3/100
455/455 [==============================] - 265s 582ms/step - loss: 1.4992 - accuracy: 0.4271 - val_loss: 1.3255 - val_accuracy: 0.4826

Epoch 00003: val_accuracy improved from 0.43550 to 0.48259, saving model to emotions3.h5
Epoch 4/100
455/455 [==============================] - 279s 613ms/step - loss: 1.3965 - accuracy: 0.4660 - val_loss: 1.2676 - val_accuracy: 0.5093

Epoch 00004: val_accuracy improved from 0.48259 to 0.50933, saving model to emotions3.h5
Epoch 5/100
455/455 [==============================] - 